# Introduction

This notebooks presents **ConvNet** in PyTorch used to solve **Street View House Numbers** task.

This is replication of _Multi-digit Number Recognition from Street View Imagery using Deep Convolutional Neural Networks_

**Contents**
* [Imports](#Imports)
* [Dataset](#Dataset)
* [Model](#Model)

# Config

Download and extract [SVHN](http://ufldl.stanford.edu/housenumbers/) dataset in **Format 1** (train.tar.gz, test.tar.gz, extra.tar.gz)

In [1]:
dataset_location = '/home/marcin/Datasets/SVHN'  # .../train/1.png

# Imports

In [16]:
import os
import pathlib

import numpy as np

import h5py  # required to open .mat files in SVHN dataset

In [3]:
import torch
import torch.nn as nn

# Dataset

In [4]:
dataset_path = pathlib.Path(dataset_location)
assert os.path.isfile(dataset_path / 'extra/1.png')
assert os.path.isfile(dataset_path / 'train/1.png')
assert os.path.isfile(dataset_path / 'test/1.png')

In [35]:
index = 29507
with h5py.File(dataset_path / 'test/digitStruct.mat') as f:
    item = f['digitStruct']['bbox'][0].item()
    ho = f[item]['label'][0]
    
    aaa = f['digitStruct']['bbox'][0]['label']
    #ho = data = f.get('digitStruct/bbox')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [36]:
f = h5py.File(dataset_path / 'test/digitStruct.mat')

In [37]:
list(f.keys())

['#refs#', 'digitStruct']

In [138]:
index = 0

In [130]:
dset_name = f['/digitStruct/name']
dset_name

<HDF5 dataset "name": shape (13068, 1), type "|O">

In [131]:
dset_name.size

13068

In [132]:
ref = dset_name[index][0]

In [133]:
''.join(chr(v[0]) for v in f[ref].value)

'2.png'

In [243]:
class DigitDesc:
    def __init__(self, label, left, top, width, height):
        assert label == int(label)
        assert left == int(left)
        assert top == int(top)
        assert width == int(width)
        assert height == int(height)
        
        self.label = int(label)
        self.left = int(left)
        self.top = int(top)
        self.width = int(width)
        self.height = int(height)
    
    def __str__(self):
        return f'{self.label}:{self.left},{self.top},{self.width},{self.height}'

In [304]:
index = 1

In [305]:
labels = []

In [306]:
dset_bbox = f['/digitStruct/bbox']
dset_bbox

<HDF5 dataset "bbox": shape (13068, 1), type "|O">

In [307]:
ref = dset_bbox[index][0]
ref

<HDF5 object reference>

In [308]:
ref = f['digitStruct']['bbox'][index].item()
ref

<HDF5 object reference>

In [314]:
def read_digits(f, index):
    
    ref = f['digitStruct']['bbox'][index].item()
    
    ddd = {}
    for key in ['label', 'left', 'top', 'width', 'height']:
        dset = f[ref][key]
        if len(dset) == 1:
            ddd[key] = [ int(dset[0][0]) ]
        else:
            ddd[key] = []
            for i in range(len(dset)):
                ref2 = dset[i][0]
                ddd[key].append( int(f[ref2][0][0]) )
    return ddd

In [321]:
read_digits(f, 6)

{'label': [1, 8, 3],
 'left': [44, 51, 62],
 'top': [7, 6, 6],
 'width': [9, 11, 10],
 'height': [21, 21, 21]}

In [313]:
dd

{'label': [2, 1, 10],
 'left': [99, 114, 121],
 'top': [5, 8, 6],
 'width': [14, 8, 12],
 'height': [23, 23, 23]}

In [311]:
ref2

<HDF5 object reference>

In [272]:
def read_digit(f, ref):
    label = f[ref]['label'][0, 0]
    left = f[ref]['left'][0][0]
    top = f[ref]['top'][0][0]
    width = f[ref]['width'][0][0]
    height = f[ref]['height'][0][0]
    digit = DigitDesc(label, left, top, width, height)
    return digit

In [277]:

obj = f[ref]['label']
if obj.shape[0] == 1:
    # single digit on image
    digit = read_digit(f, ref)
    labels.append(digit)
else:
    # multiple digits on image
    for i in range(obj.shape[0]):
        ref2 = obj[i][0]
        label = f[ref2][0][0]
        labels.append(int(label))

In [281]:
labels

[2, 1, 10, 2, 1, 10]

In [278]:
f[ref2][0][0]

10.0

In [279]:
ii = 0
ref2 = xx[ii][0]

In [280]:
f[ref2][0][0]

AttributeError: 'numpy.float64' object has no attribute 'encode'

In [141]:
left = f[ref]['left'][0][0]
top = f[ref]['top'][0][0]
width = f[ref]['width'][0][0]
height = f[ref]['height'][0][0]

In [142]:
print(left, top, width, height)

43.0 7.0 19.0 30.0


In [128]:
f[ref]['label'][0]

array([5.])

In [ ]:
a